## Loading our data

We'll be using the same USGS water dataset as lesson 3.  Here we read and pre-process the data.

In [30]:
import pandas as pd

In [31]:
# Read in the data
water_vars = pd.read_csv('../../data/englewood_3_12_21_usgs_water.tsv', sep='\t', skiprows=30)
# There are a lot of variables here, so let's shorten our dataframe to a few variables
water_vars = water_vars[['datetime', '210920_00060', '210922_00010', '210924_00300', '210925_00400']]
# Get rid of the first row of hard-coded datatype info
water_vars = water_vars.drop(0)
# Rename the columns from their USGS codes to more human-readible names
name_codes = {'210920_00060': 'discharge','210922_00010': 'temperature', '210924_00300': 'dissolved oxygen', '210925_00400': 'pH'}
water_vars = water_vars.rename(columns=name_codes)
# Convert columns with numbers to a numeric type
water_vars['discharge'] = pd.to_numeric(water_vars['discharge'])
water_vars['temperature'] = pd.to_numeric(water_vars['temperature'])
water_vars['dissolved oxygen'] = pd.to_numeric(water_vars['dissolved oxygen'])
water_vars['pH'] = pd.to_numeric(water_vars['pH'])

Today I'm also going to add two fake columns of data, "dam release" and "safety level", that will help us as we go through some of the new concepts.

In [32]:
import random

In [33]:
water_vars['dam release'] = [random.choice([True, False]) for x in range(len(water_vars))]
water_vars['safety level'] = [random.choice(['low', 'medium', 'high']) for x in range(len(water_vars))]

In [34]:
water_vars

,datetime,discharge,temperature,dissolved oxygen,pH,dam release,safety level
1,2021-03-12 00:00,44.5,8.1,8.3,8.1,False,low
2,2021-03-12 00:15,44.5,8.1,8.2,8.1,True,low
3,2021-03-12 00:30,44.5,8.1,8.2,8.1,True,high
4,2021-03-12 00:45,44.5,8.1,8.1,8.1,False,medium
5,2021-03-12 01:00,44.5,8.1,8.1,8.1,True,medium
...,...,...,...,...,...,...,...
142,2021-03-13 11:15,42.6,6.7,9.8,7.9,False,low
143,2021-03-13 11:30,42.6,6.7,9.9,7.9,False,low
144,2021-03-13 11:45,42.6,6.7,10.2,7.9,True,medium
145,2021-03-13 12:00,46.5,6.7,10.3,7.9,False,high


## Filtering

In [60]:
# https://www.listendata.com/2019/07/how-to-filter-pandas-dataframe.html

### `loc` vs. `iloc`

# Groupby

While we have 146 individual readings in our dataset sometimes we don't care about each individual reading.  Instead we care about the aggregate of a group of readings. 

For example:
* What is the average temperature in each state, given the average temperature of every county in the US?
* How many KFCs were opened each year, given a list of the opening dates of every KCF? 🐓

In pandas (and tabular data in general) we do that with `groupby`.

## Examples

Our general sytax is:
`-dataframe-.groupby(-grouping-).-aggregation-`

TODO make this nicer to look at ^

In [39]:
water_vars.groupby("safety level").count()

,datetime,discharge,temperature,dissolved oxygen,pH,dam release
safety level,,,,,,
high,48,48,48,48,48,48
low,46,46,46,46,46,46
medium,52,51,50,52,52,52


How can it be that temperature and datetime have a different number of measurements?

There are lots of aggretation functions.  Common ones include:
* `.count()` - find the total number of rows
* `.min()`- find the minimum value of those rows
* `.max()` - find the maximum value of those rows
* `.mean()`- find the mean value of those rows
* `.sum()` - find the sum of the values of those rows

Since sometimes the dataframes can be large, I will often specify which column I want to do the aggregation on when I do the groupby.

In [40]:
# Group our water dataframe by saftey level and take the mean of the discharge values for each safety level group
water_vars.groupby("safety level")['discharge'].mean()

safety level
high      44.433333
low       45.160870
medium    44.831373
Name: discharge, dtype: float64

### 📝 Checking In
Find the max pH value for each type of safety level.

Find the mean discharge during dam releases.

## Breaking down the process
split -- apply -- combine

https://static.packt-cdn.com/products/9781783985128/graphics/5128OS_09_01.jpg



## Variations on the idea

### Group by multiple fields

In [44]:
water_vars.groupby(['safety level', 'dam release'])['pH'].mean()

safety level  dam release
high          False          8.008000
              True           7.965217
low           False          7.971429
              True           8.000000
medium        False          8.010345
              True           7.973913
Name: pH, dtype: float64

### 📝 Checking In
Explain the above output.

...

### Aggregating multiple columns

In [42]:
water_vars.groupby("dam release").agg(
    max_methane=pd.NamedAgg(column='discharge', aggfunc='max'), 
    mean_dms=pd.NamedAgg(column="dissolved oxygen", aggfunc='mean')
)

,max_methane,mean_dms
dam release,,
False,48.5,9.596341
True,48.5,9.157812


### 📝 Checking In
Explain the output

Find the minimum temperature value and mean discharge for each safety level.

Find the minimum temperature value and mean discharge for each safety level and dam release.

### Binning a numeric value to split on

In [45]:
water_vars['temperature'].describe()

count    144.000000
mean       7.512500
std        0.571451
min        6.600000
25%        6.900000
50%        7.600000
75%        8.000000
max        8.500000
Name: temperature, dtype: float64

In [49]:
bin_intervals = pd.IntervalIndex.from_tuples([(6, 7), (7, 8), (8, 9)])
print(bin_intervals)

IntervalIndex([(6, 7], (7, 8], (8, 9]],
              closed='right',
              dtype='interval[int64]')


"closed right" means that the right value is included in that bin.  Another way to think about it is that the `]` means less than or equal to `<=` while the `(` bracket means less than `<`.
So our data values range from 
* bin 1: 6 < x <=7
* bin 2: 7 < x <= 8
* bin 3: 8 < x <= 9

If there are values outside your bins when you apply them they will be dropped

In [56]:
# TODO why aren't the labels applied?
bins = pd.cut(water_vars['temperature'], bins=bin_intervals, labels=['cold', 'moderate', 'warm'])

In [57]:
bins

1      (8, 9]
2      (8, 9]
3      (8, 9]
4      (8, 9]
5      (8, 9]
        ...  
142    (6, 7]
143    (6, 7]
144    (6, 7]
145    (6, 7]
146    (6, 7]
Name: temperature, Length: 146, dtype: category
Categories (3, interval[int64]): [(6, 7] < (7, 8] < (8, 9]]

In [59]:
water_vars.groupby(bins)['dissolved oxygen'].mean()

temperature
(6, 7]     9.169767
(7, 8]     8.822857
(8, 9]    11.125806
Name: dissolved oxygen, dtype: float64

### 📝 Checking In
Same setup with bins

Multiple aggregation